In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np


def log_reader(log_name: str) -> pd.DataFrame:
    """
    逐行读取日志，分割并提取，最后保存为 pandas dataframe
    """
    with open(log_name, 'r', encoding='cp1252') as file:
        # Get the total number of lines in the file
        total_lines = sum(1 for _ in file)

    with open(log_name, 'r', encoding='cp1252') as file:
        split_results = []
        # for line in file:
        for line in tqdm(file, total=total_lines, desc="Reading logs by lines:"):
            words = line.split()[4:]
            word_pair_dict = {}
            for pairs in words:
                try:
                    ls_pairs = pairs.split("=")
                    word_pair_dict[ls_pairs[0]] = ls_pairs[1]
                except IndexError:
                    # ls_pairs = pairs.split("=")
                    # word_pair_dict[ls_pairs[0]] = ""
                    pass
            split_results.append(word_pair_dict)

    results_df = pd.DataFrame(split_results)
    return results_df


def count_column_with_ratios(df_name: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    根据列名分组，分组后计数，计数后进行降序排序
    """
    count_col_name = col_name + "计数"
    col_counts_df = df_name.groupby(col_name).size().reset_index(name=count_col_name)
    sorted_df = col_counts_df.sort_values(by=count_col_name, ascending=False)
    total_counts = sorted_df[count_col_name].sum()
    sorted_df[col_name + "%比"] = sorted_df[count_col_name].apply(lambda x: (round(x / total_counts, 4)) * 100)
    return sorted_df


def select_by_column_name_and_value(df_name: pd.DataFrame, col_name: str, col_value) -> pd.DataFrame:
    """
    选取某行是某值的所有数据
    :param df_name:
    :param col_name:
    :param col_value:
    :return:
    """
    filter_df = df_name.loc[df_name[col_name] == col_value]
    return filter_df


def analysis_selected_dataframe(selected_df: pd.DataFrame) -> pd.DataFrame:
    """
    解析Dataframe表格中的 "service", "srcport", "dstport", "srcip", "dstip" 中的五项指标的计数排名及百分比
    :param selected_df:
    :return:
    """
    # ls_cols=["service", "srcport", "dstport", "srcip", "dstip", "dstcountry"]:
    ls_cols = ["service", "srcport", "dstport", "srcip", "dstip"]
    total_column_list = ['default_1', 'default_2', 'default_3']
    total_ndarray = np.empty((20, 3))
    for col in ls_cols:
        selected_df_col_counts = count_column_with_ratios(selected_df, col).head(20)
        selected_df_col_counts_columns = selected_df_col_counts.columns.tolist()
        selected_df_col_counts_ndarray = selected_df_col_counts.to_numpy()
        total_column_list = total_column_list + selected_df_col_counts_columns
        total_ndarray = np.concatenate((total_ndarray, selected_df_col_counts_ndarray), axis=1)  # 注意指定axis=1以进行横向拼接
    total_dataframe = pd.DataFrame(total_ndarray, columns=total_column_list)
    total_dataframe = total_dataframe.drop(['default_1', 'default_2', 'default_3'], axis=1)
    return total_dataframe


def detail_analysis_by_column_and_value(total_logs_df: pd.DataFrame, col_name: str, col_value):
    """
    解析Dataframe表格中的 "service", "srcport", "dstport", "srcip", "dstip" 中的五项指标的计数排名及百分比
    以及 srccountry 和 dstcountry 2项指标的计数排名及百分比
    并存储为 execl表格
    :param total_logs_df:
    :param col_name:
    :param col_value:
    :return:
    """
    selected_df_by_column_name_and_value = select_by_column_name_and_value(total_logs_df, col_name, col_value)
    detail_result_total_df = analysis_selected_dataframe(selected_df_by_column_name_and_value)
    output_execl_name = ("LongLogsAnalysisResultsTables\\" + col_name + col_value.strip("\"") + "DetailAnalysisResults.xlsx")
    detail_result_total_df.to_excel(output_execl_name, index=False)
    print("DataFrame saved to " + output_execl_name)

    srccountry_col_counts_df = (count_column_with_ratios(selected_df_by_column_name_and_value, 'srccountry').head(20))
    output_execl_name = "LongLogsAnalysisResultsTables\\" + col_name + col_value.strip("\"") + "srccountryResults.xlsx"
    srccountry_col_counts_df.to_excel(output_execl_name, index=False)
    print("DataFrame saved to " + output_execl_name)

    dstcountry_col_counts_df = (count_column_with_ratios(selected_df_by_column_name_and_value, 'dstcountry').head(20))
    output_execl_name = "LongLogsAnalysisResultsTables\\" + col_name + col_value.strip("\"") + "dstcountryResults.xlsx"
    dstcountry_col_counts_df.to_excel(output_execl_name, index=False)
    print("DataFrame saved to " + output_execl_name)

    return detail_result_total_df, srccountry_col_counts_df, dstcountry_col_counts_df


if __name__ == '__main__':
    the_log_short = "message_179.170.130.210.bn.2iij.net_20230930.log"
    the_log_long = "message_179.170.130.210.bn.2iij.net_20231002.log"
    the_log_long_1031 = "message_179.170.130.210.bn.2iij.net_20231031.log"
    the_log_long_1208 = "message_179.170.130.210.bn.2iij.net_20231210.log"
    all_logs_df = log_reader(the_log_long_1208)

Reading logs by lines:: 100%|██████████| 119366/119366 [00:01<00:00, 71494.72it/s]


In [7]:
# important
# AllLogidRatios: Logid, Logid计数， Logid百分比
selected_df_col_counts = count_column_with_ratios(all_logs_df,"logid")
print("index = 0 value = " + all_logs_df.iloc[0]['logid'])
print("index = 1 value = " + all_logs_df.iloc[1]['logid'])
print("index = 131 value = " + all_logs_df.iloc[131]['logid'])
print("index = 220 value = " + all_logs_df.iloc[220]['logid'])
selected_df_col_counts

index = 0 value = "0000000013"
index = 1 value = "0001000014"
index = 131 value = "0000000011"
index = 220 value = "0000000020"


,logid,logid计数,logid%比
3,"""0001000014""",51688,43.30
1,"""0000000013""",49771,41.70
2,"""0000000020""",14447,12.10
0,"""0000000011""",2736,2.29
8,"""0100040704""",288,0.24
13,"""0101039946""",164,0.14
10,"""0101039426""",138,0.12
7,"""0100026003""",48,0.04
5,"""0100026001""",26,0.02
12,"""0101039944""",22,0.02


In [23]:
# important
# AllLogidRatios: action, action计数， action百分比
df_logid_xxx = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[220]['logid']) 
print(count_column_with_ratios(df_logid_xxx, "action").head(20))

      action  action计数  action%比
1  "ip-conn"     32896      79.1
0      "dns"      8694      20.9


In [19]:
# Importances
# LogLevelRatios
df_col_service_counts = count_column_with_ratios(all_logs_df, "level").head(20)
df_col_service_counts

,level,level计数,level%比
2,"""notice""",711302,94.39
3,"""warning""",41590,5.52
1,"""information""",597,0.08
0,"""alert""",85,0.01


In [14]:
df_logid_xxx = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[0]['logid']) 
print(count_column_with_ratios(df_logid_xxx, "type").head(20))
print(count_column_with_ratios(df_logid_xxx, "subtype").head(20))
print(count_column_with_ratios(df_logid_xxx, "level").head(20))
print(count_column_with_ratios(df_logid_xxx, "msg").head(20))
print(count_column_with_ratios(df_logid_xxx, "logdesc").head(20))
print(count_column_with_ratios(df_logid_xxx, "desc").head(20))
print(count_column_with_ratios(df_logid_xxx, "reason").head(20))

        type  type计数  type%比
0  "traffic"  104194   100.0
     subtype  subtype计数  subtype%比
0  "forward"     104194      100.0
      level  level计数  level%比
0  "notice"   104194    100.0
Empty DataFrame
Columns: [msg, msg计数, msg%比]
Index: []
Empty DataFrame
Columns: [logdesc, logdesc计数, logdesc%比]
Index: []
Empty DataFrame
Columns: [desc, desc计数, desc%比]
Index: []
Empty DataFrame
Columns: [reason, reason计数, reason%比]
Index: []


In [18]:
detail_analysis_by_column_and_value(all_logs_df, 'logid', all_logs_df.iloc[220]['logid'])

DataFrame saved to LongLogsAnalysisResultsTables\logid0000000011DetailAnalysisResults.xlsx
DataFrame saved to LongLogsAnalysisResultsTables\logid0000000011srccountryResults.xlsx
DataFrame saved to LongLogsAnalysisResultsTables\logid0000000011dstcountryResults.xlsx


(        service service计数 service%比 srcport srcport计数 srcport%比 dstport  \
 0       "HTTPS"     29965     72.05   16403        37      0.09     443   
 1         "DNS"      9575     23.02   56166        11      0.03      53   
 2     "udp/443"       928      2.23   50640        10      0.02      80   
 3        "HTTP"       477      1.15   60781        10      0.02    5223   
 4    "icmp/0/8"       259      0.62   55511        10      0.02   17472   
 5    "tcp/5223"        64      0.15   51965         9      0.02    8740   
 6   "tcp/17472"        62      0.15   52147         9      0.02   10001   
 7    "tcp/8740"        18      0.04   55471         9      0.02    8081   
 8   "tcp/10001"        17      0.04   54222         9      0.02     853   
 9    "tcp/8081"        16      0.04   55429         9      0.02    3478   
 10    "tcp/853"        15      0.04   50427         9      0.02    8080   
 11   "udp/3478"        12      0.03   55102         9      0.02      22   
 12   "tcp/8

分割线

In [ ]:
df_col_level_val_notice = select_by_column_name_and_value(all_logs_df, "level", "\"notice\"")
df_col_level_val_notice_count_service = count_column_with_ratios(df_col_level_val_notice, "service").head(20)
df_col_level_val_notice_count_service.head(20)

In [ ]:
df_col_service_val_https = select_by_column_name_and_value(all_logs_df, "service", "\"HTTPS\"")
df_col_service_val_https.head(20)

In [ ]:
df_col_service_val_https_counts_service = count_column_with_ratios(df_col_service_val_https, "dstcountry").head(20)
df_col_service_val_https_counts_service

In [ ]:
df_col_type_val_traffic_counts_service = count_column_with_ratios(df_col_type_val_traffic, "reason").head(20)
df_col_type_val_traffic_counts_service

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "level").head(20)
df_col_service_counts

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "service").head(20)
df_col_service_counts

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "srcintf").head(20)
df_col_service_counts

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "srcintfrole").head(20)
df_col_service_counts

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "dstintf").head(20)
df_col_service_counts

In [ ]:
df_col_service_counts = count_column_with_ratios(all_logs_df, "appcat").head(20)
df_col_service_counts

In [ ]:
df_logid_13 = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[0]['logid']) 
df_col_service_counts = count_column_with_ratios(df_logid_13, "subtype").head(20)
df_col_service_counts

In [ ]:
df_logid_20 = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[1]['logid']) 
df_col_service_counts = count_column_with_ratios(df_logid_20, "subtype").head(20)
df_col_service_counts

In [ ]:
df_logid_14 = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[53]['logid']) 
df_col_service_counts = count_column_with_ratios(df_logid_14, "subtype").head(20)
df_col_service_counts

In [ ]:
df_logid_11 = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[12]['logid']) 
df_col_service_counts = count_column_with_ratios(df_logid_11, "subtype").head(20)
df_col_service_counts

In [ ]:
detail_analysis_by_column_and_value(all_logs_df, 'logid', all_logs_df.iloc[12]['logid'])

In [ ]:
print(all_logs_df.columns)
print("index = 0 value = " + all_logs_df.iloc[0]['logid'])
print("index = 1 value = " + all_logs_df.iloc[1]['logid'])
print("index = 12 value = " + all_logs_df.iloc[12]['logid'])
print("index = 53 value = " + all_logs_df.iloc[53]['logid'])
print("index = 827 value = " + all_logs_df.iloc[827]['logid'])
print("index = 1399 value = " + all_logs_df.iloc[1399]['logid'])
print("index = 3611 value = " + all_logs_df.iloc[3611]['logid'])
print("index = 15150 value = " + all_logs_df.iloc[15150]['logid'])
print("index = 100068 value = " + all_logs_df.iloc[100068]['logid'])
print("index = 612439 value = " + all_logs_df.iloc[612439]['logid'])
print("index = 613245 value = " + all_logs_df.iloc[613245]['logid'])
print("index = 756415 value = " + all_logs_df.iloc[756415]['logid'])
print("index = 913468 value = " + all_logs_df.iloc[913468]['logid'])
print("index = 921527 value = " + all_logs_df.iloc[921527]['logid'])
print("index = 804075 value = " + all_logs_df.iloc[804075]['logid'])
print("index = 984182 value = " + all_logs_df.iloc[984182]['logid'])

In [ ]:
df_logid_xxx = select_by_column_name_and_value(all_logs_df, 'logid', all_logs_df.iloc[984182]['logid']) 
print(count_column_with_ratios(df_logid_xxx, "type").head(20))
print(count_column_with_ratios(df_logid_xxx, "subtype").head(20))
print(count_column_with_ratios(df_logid_xxx, "level").head(20))
print(count_column_with_ratios(df_logid_xxx, "msg").head(20))
print(count_column_with_ratios(df_logid_xxx, "logdesc").head(20))
print(count_column_with_ratios(df_logid_xxx, "desc").head(20))
print(count_column_with_ratios(df_logid_xxx, "reason").head(20))

In [8]:
import numpy as np

# 指定期望的形状
desired_shape = (3, 4)

# 创建一个包含数据的2D NumPy数组
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 使用np.pad函数进行填充，以确保形状匹配
padded_data = np.pad(data, ((0, max(0, desired_shape[0] - data.shape[0])), (0, max(0, desired_shape[1] - data.shape[1]))), mode='constant', constant_values=0)

# 打印结果
print("原始数据:")
print(data)
print("\n填充后的数据:")
print(padded_data)

原始数据:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

填充后的数据:
[[1 2 3 0]
 [4 5 6 0]
 [7 8 9 0]]


In [9]:
import numpy as np

# 期望的行数
desired_rows = 4

# 原始数据
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 如果当前行数小于期望的行数，使用resize函数填充零行
if data.shape[0] < desired_rows:
    padded_data = np.resize(data, (desired_rows, data.shape[1]))
else:
    padded_data = data.copy()

# 打印结果
print("原始数据:")
print(data)
print("\n填充后的数据:")
print(padded_data)


原始数据:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

填充后的数据:
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [1 2 3]]


In [10]:
import numpy as np

# 期望的行数
desired_rows = 4

# 原始数据
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# 计算需要填充的行数
padding_rows = max(0, desired_rows - data.shape[0])

# 使用pad函数在底部填充零行
padded_data = np.pad(data, ((0, padding_rows), (0, 0)), mode='constant', constant_values=0)

# 打印结果
print("原始数据:")
print(data)
print("\n填充后的数据:")
print(padded_data)

原始数据:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

填充后的数据:
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [0 0 0]]
